Wybór odpowiedniej bazy danych

In [ ]:
use u_szybinsk;

Usuwanie istniejących danych

In [ ]:
-- drop data
DECLARE @sql NVARCHAR(max)=''

-- Drop data: constraints
SELECT @sql += 'ALTER TABLE ' + QUOTENAME(OBJECT_SCHEMA_NAME(parent_object_id))
    + '.' + QUOTENAME(OBJECT_NAME(parent_object_id)) + 
    ' DROP CONSTRAINT ' + QUOTENAME(name) + ';'
FROM sys.foreign_keys;

EXEC sp_executesql @sql
SET @sql = ''

-- Drop data: tables
SELECT @sql += 'DROP TABLE ' + QUOTENAME(s.NAME) + '.' + QUOTENAME(t.NAME) + '; '
FROM sys.tables t
JOIN sys.schemas s ON t.[schema_id] = s.[schema_id]
WHERE t.type = 'U'

EXEC sp_executesql @sql
SET @sql = ''

-- Drop data: procedures
SELECT @sql += 'DROP PROCEDURE ' + QUOTENAME(SCHEMA_NAME(p.schema_id)) + '.' + QUOTENAME(p.NAME) + ';'
FROM sys.procedures p

EXEC sp_executesql @sql
SET @sql = ''

-- Drop data: views
SELECT @sql += 'DROP VIEW ' + QUOTENAME(SCHEMA_NAME(schema_id)) + '.' + QUOTENAME(v.name) +';'
FROM sys.views v

EXEC sp_executesql @sql

-- Drop data: types
SELECT @sql += 'DROP TYPE ' + QUOTENAME(SCHEMA_NAME(schema_id)) + '.' + QUOTENAME(name)
FROM sys.types
WHERE is_user_defined = 1

-- Drop data: roles
DROP ROLE RESTAURANT_OWNER;
DROP ROLE MANAGER;
DROP ROLE CHEF;
DROP ROLE WAITER;
DROP ROLE CLIENT;

EXEC sp_executesql @sql

Tworzenie tabel

In [ ]:
-- tables
-- Table: Clients
CREATE TABLE Clients (
    client_id int  NOT NULL IDENTITY(1,1),
    email varchar(30)  NULL UNIQUE,
    phone varchar(20)  NULL UNIQUE,
    address varchar(30)  NOT NULL,
    postcode varchar(10)  NOT NULL,
    city_id int  NOT NULL,
    CONSTRAINT Clients_pk PRIMARY KEY  (client_id)
);

-- Table: Individual_Clients
CREATE TABLE Individual_Clients (
    client_id int  NOT NULL,
    first_name varchar(20)  NOT NULL,
    last_name varchar(20)  NOT NULL,
    CONSTRAINT Individual_Clients_pk PRIMARY KEY  (client_id)
);

-- Table: Company_Clients
CREATE TABLE Company_Clients (
    client_id int  NOT NULL,
    name varchar(20)  NOT NULL UNIQUE,
    NIP varchar(15)  NULL UNIQUE,
    CONSTRAINT Company_Clients_pk PRIMARY KEY  (client_id)
);

-- Table: Countries
CREATE TABLE Countries (
    country_id int  NOT NULL IDENTITY(1,1),
    country_name varchar(30)  NOT NULL UNIQUE,
    CONSTRAINT Countries_pk PRIMARY KEY  (country_id)
);

-- Table: Cities
CREATE TABLE Cities (
    city_id int  NOT NULL IDENTITY(1,1),
    city_name varchar(30)  NOT NULL UNIQUE,
    country_id int  NOT NULL,
    CONSTRAINT Cities_pk PRIMARY KEY  (city_id)
);

-- Table: Discounts
CREATE TABLE Discounts (
    client_id int  NOT NULL,
    discount float(2)  NOT NULL,
    start_date date  NOT NULL,
    end_date date  NOT NULL,
    CONSTRAINT Discounts_pk PRIMARY KEY  (client_id,start_date),
    CONSTRAINT Discounts_ck_discount CHECK  (discount > 0 AND discount <= 1),
    CONSTRAINT Discounts_ck_end_date CHECK  (end_date > start_date)
);

-- Table: Products
CREATE TABLE Products (
    product_id int  NOT NULL IDENTITY(1,1),
    product_name varchar(30)  NOT NULL UNIQUE,
    is_sea_food bit  NOT NULL,
    available_every_day int  NOT NULL,
    available_today int  NOT NULL,
    CONSTRAINT Products_pk PRIMARY KEY  (product_id),
    CONSTRAINT Products_ch_available_today CHECK  (available_today >= 0 AND available_today <= available_every_day)
);

-- Table: Menu
CREATE TABLE Menu (
    menu_id int  NOT NULL IDENTITY(1,1),
    start_date date  NOT NULL,
    end_date date  NOT NULL,
    CONSTRAINT Menu_pk PRIMARY KEY  (menu_id),
    CONSTRAINT Menu_ck_end_date CHECK  (end_date > start_date)
);

-- Table: Menu_Details
CREATE TABLE Menu_Details (
    menu_id int  NOT NULL,
    product_id int  NOT NULL,
    price money  NOT NULL,
    is_active bit  NOT NULL,
    CONSTRAINT Menu_Details_pk PRIMARY KEY  (menu_id,product_id),
    CONSTRAINT Menu_Details_ck_price CHECK  (price > 0)
);

-- Table: Reservation_Details
CREATE TABLE Reservation_Details (
    reservation_id int  NOT NULL,
    product_id int  NOT NULL,
    quantity int  NOT NULL,
    price money  NOT NULL,
    CONSTRAINT Reservation_Details_pk PRIMARY KEY  (reservation_id,product_id),
    CONSTRAINT Reservation_Details_ck_quantity CHECK  (quantity > 0),
    CONSTRAINT Reservation_Details_ck_price CHECK  (price > 0)
);

-- Table: Reservation_People
CREATE TABLE Reservation_People (
    reservation_id int  NOT NULL,
    last_name varchar(20)  NOT NULL,
    CONSTRAINT Reservation_People_pk PRIMARY KEY  (reservation_id,last_name)
);

-- Table: Reservation_Tables
CREATE TABLE Reservation_Tables (
    reservation_id int  NOT NULL,
    table_id int  NOT NULL,
    CONSTRAINT Reservation_Tables_pk PRIMARY KEY  (reservation_id,table_id)
);

-- Table: Reservations
CREATE TABLE Reservations (
    reservation_id int  NOT NULL IDENTITY(1,1),
    reservation_date datetime  NOT NULL,
    target_date datetime  NULL,
    eating_time datetime  NULL,
    actual_eating_time datetime  NULL,
    people_count int  NULL,
    client_id int  NULL,
    discount float(2)  NOT NULL,
    invoice_id int  NULL,
    CONSTRAINT Reservations_pk PRIMARY KEY  (reservation_id),
    CONSTRAINT Reservations_ck_eating_time CHECK  (eating_time > target_date),
    CONSTRAINT Reservations_ck_people_count CHECK  (people_count > 0 AND people_count <= 50),
    CONSTRAINT Reservations_ck_discount CHECK  (discount >= 0 AND discount <= 1)
);

-- Table: Tables
CREATE TABLE Tables (
    table_id int  NOT NULL IDENTITY(1,1),
    capacity int  NOT NULL,
    is_active bit  NOT NULL,
    CONSTRAINT Tables_pk PRIMARY KEY  (table_id),
    CONSTRAINT Tables_ck_capacity CHECK  (capacity > 0)
);

-- Table: Invoices
CREATE TABLE Invoices (
    invoice_id int  NOT NULL IDENTITY(1,1),
    invoice_number varchar(30)  NOT NULL UNIQUE,
    created datetime  NOT NULL,
    is_payed bit  NOT NULL,
    address varchar(30)  NOT NULL,
    postcode varchar(10)  NOT NULL,
    city_id int  NOT NULL,
    CONSTRAINT Invoices_pk PRIMARY KEY  (invoice_id)
);

-- foreign keys
-- Reference: Clients_Cities (table: Clients)
ALTER TABLE Clients ADD CONSTRAINT Clients_Cities
    FOREIGN KEY (city_id)
    REFERENCES Cities (city_id);

-- Reference: Company_Clients_Clients (table: Company_Clients)
ALTER TABLE Company_Clients ADD CONSTRAINT Company_Clients_Clients
    FOREIGN KEY (client_id)
    REFERENCES Clients (client_id);

-- Reference: Countries_Cities (table: Cities)
ALTER TABLE Cities ADD CONSTRAINT Countries_Cities
    FOREIGN KEY (country_id)
    REFERENCES Countries (country_id);

-- Reference: Discounts_Clients (table: Discounts)
ALTER TABLE Discounts ADD CONSTRAINT Discounts_Clients
    FOREIGN KEY (client_id)
    REFERENCES Clients (client_id);

-- Reference: Individual_Clients_Clients (table: Individual_Clients)
ALTER TABLE Individual_Clients ADD CONSTRAINT Individual_Clients_Clients
    FOREIGN KEY (client_id)
    REFERENCES Clients (client_id);

-- Reference: Menu_Details_Products (table: Menu_Details)
ALTER TABLE Menu_Details ADD CONSTRAINT Menu_Details_Products
    FOREIGN KEY (product_id)
    REFERENCES Products (product_id);

-- Reference: Menu_Menu_Details (table: Menu_Details)
ALTER TABLE Menu_Details ADD CONSTRAINT Menu_Menu_Details
    FOREIGN KEY (menu_id)
    REFERENCES Menu (menu_id);

-- Reference: Reservation_Details_Products (table: Reservation_Details)
ALTER TABLE Reservation_Details ADD CONSTRAINT Reservation_Details_Products
    FOREIGN KEY (product_id)
    REFERENCES Products (product_id);

-- Reference: Reservation_People_Reservations (table: Reservation_People)
ALTER TABLE Reservation_People ADD CONSTRAINT Reservation_People_Reservations
    FOREIGN KEY (reservation_id)
    REFERENCES Reservations (reservation_id);

-- Reference: Reservations_Clients (table: Reservations)
ALTER TABLE Reservations ADD CONSTRAINT Reservations_Clients
    FOREIGN KEY (client_id)
    REFERENCES Clients (client_id);

-- Reference: Reservations_Invoices (table: Reservations)
ALTER TABLE Reservations ADD CONSTRAINT Reservations_Invoices
    FOREIGN KEY (invoice_id)
    REFERENCES Invoices (invoice_id);

-- Reference: Reservations_Reservation_Details (table: Reservation_Details)
ALTER TABLE Reservation_Details ADD CONSTRAINT Reservations_Reservation_Details
    FOREIGN KEY (reservation_id)
    REFERENCES Reservations (reservation_id);

-- Reference: Reservations_Reservation_Tables (table: Reservation_Tables)
ALTER TABLE Reservation_Tables ADD CONSTRAINT Reservations_Reservation_Tables
    FOREIGN KEY (reservation_id)
    REFERENCES Reservations (reservation_id);

-- Reference: Tables_Reservation_Tables (table: Reservation_Tables)
ALTER TABLE Reservation_Tables ADD CONSTRAINT Tables_Reservation_Tables
    FOREIGN KEY (table_id)
    REFERENCES Tables (table_id);

Tworzenie procedur

In [ ]:
-- procedures
-- Procedure: add_individual_client
-- Procedura dodaje nowego klienta indywidualnego
CREATE PROCEDURE add_individual_client @email VARCHAR(30), @phone VARCHAR(20), @first_name VARCHAR(20), @last_name VARCHAR(20), @address VARCHAR(30), @postcode VARCHAR(10), @city_id INT
AS 
    BEGIN TRAN
    BEGIN TRY
        INSERT INTO Clients(email, phone, address, postcode, city_id)  
        VALUES (@email, @phone, @address, @postcode, @city_id)

        INSERT INTO Individual_Clients (client_id, first_name, last_name)  
        VALUES (@@IDENTITY, @first_name, @last_name)
    
        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: add_company_clients
-- Procedura dodaje nowe klienta biznesowego
CREATE PROCEDURE add_company_client @email VARCHAR(30), @phone VARCHAR(20), @name VARCHAR(20), @NIP VARCHAR(15), @address VARCHAR(30), @postcode VARCHAR(10), @city_id INT
AS 
    BEGIN TRAN
    BEGIN TRY
        INSERT INTO Clients(email, phone, address, postcode, city_id)
        VALUES (@email, @phone, @address, @postcode, @city_id)

        INSERT INTO Company_Clients(client_id, name, NIP)
        VALUES (@@IDENTITY, @name, @NIP)
        
        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: tables_available
-- Procedura pokazuje wolne stoliki w podanym przedziale czasowym
CREATE PROCEDURE tables_available @start_date DATETIME, @end_date DATETIME 
AS  
    SELECT table_id,capacity FROM Tables
    WHERE table_id NOT IN(
    SELECT t.table_id FROM Reservations AS r
    INNER JOIN Reservation_Tables AS rt ON r.reservation_id = rt.reservation_id
    RIGHT OUTER JOIN Tables AS t ON t.table_id = rt.table_id
    WHERE (@start_date >= r.target_date AND @start_date <=  r.eating_time) OR
      (@end_date >= r.target_date AND @end_date <=  r.eating_time) OR
      t.is_active = 0)
GO

-- Procedure: add_reservation
-- Procedura tworzy nową rezerwację
CREATE TYPE Table_List AS TABLE ( id INT NOT NULL IDENTITY(1,1),table_id INT )
GO

CREATE TYPE Name_List AS TABLE ( id INT NOT NULL IDENTITY(1,1), last_name VARCHAR(30) )
GO

CREATE PROCEDURE add_reservation @target_date DATETIME, @eating_time DATETIME, @people_count INT, @client_id INT, @last_name_list Name_List READONLY, @table_list Table_List READONLY
AS 
    BEGIN TRAN
    BEGIN TRY
        DECLARE @temp_table TABLE (table_id INT, capacity INT)
        INSERT @temp_table EXEC tables_available @target_date, @eating_time

        DECLARE @table_list_constraint INT
        SET @table_list_constraint  = (SELECT COUNT(*) FROM @table_list as tl
        where tl.table_id not in (SELECT t.table_id from @temp_table as t))

        IF(@table_list_constraint>0)
            THROW 50000, N'Podane stoliki są niedostępne', 1
        
        DECLARE @discount FLOAT(2)

        IF(@client_id in (SELECT client_id FROM Discounts))
        BEGIN
            SET @discount = (SELECT discount FROM Discounts WHERE client_id = @client_id)
        END
        ELSE
            SET @discount = 0.00

        INSERT INTO Reservations(reservation_date,target_date,eating_time,actual_eating_time,people_count,client_id,discount,invoice_id)
        VALUES (GETDATE(),@target_date,@eating_time,NULL,@people_count,@client_id,@discount,NULL)

        DECLARE @id INT
        SET @id = @@IDENTITY
        
        DECLARE @max_id_names INT = (SELECT MAX(id) FROM @last_name_list)
        DECLARE @max_id_tables INT = (SELECT MAX(id) FROM @table_list)
        DECLARE @counter INT = 1
        
        WHILE(@counter<=@max_id_names)
        BEGIN
            INSERT INTO Reservation_People(reservation_id,last_name)
            VALUES (@id,(SELECT last_name FROM @last_name_list WHERE id = @counter))
            SET @counter = @counter + 1
        END

        SET @counter = 1
        WHILE(@counter<=@max_id_tables)
        BEGIN
            INSERT INTO Reservation_Tables(reservation_id,table_id)
            VALUES (@id,(SELECT table_id FROM @table_list WHERE id = @counter))
            SET @counter = @counter + 1
        END

        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: add_reservetion_details
-- Procedura dodaje szczegóły zamówienia do rezerwacji
CREATE PROCEDURE add_reservation_details @reservation_id INT, @product_id INT, @quantity INT
AS
    BEGIN TRAN
    BEGIN TRY
        DECLARE @price MONEY
        DECLARE @menu_id INT
        DECLARE @target_date DATETIME = (SELECT target_date FROM Reservations WHERE reservation_id = @reservation_id)
        DECLARE @reservation_date DATETIME = (SELECT reservation_date FROM Reservations WHERE reservation_id = @reservation_id)

        -- Sprawdzenie czy zamówienie na owoce morza zostało wykonane odpowiednio wcześnie
        IF((SELECT is_sea_food FROM Products WHERE product_id = @product_id) = 1)
        BEGIN
            DECLARE @week_day_of_reservation_date INT = (DATEPART(WEEKDAY,@reservation_date))
            DECLARE @correct_day TABLE (day_n INT)
            INSERT INTO @correct_day VALUES(4),(5),(6)

            IF(@week_day_of_reservation_date IN (SELECT day_n FROM @correct_day))
            BEGIN
                DECLARE @week_day INT = DATEPART(WEEKDAY,@target_date)
                DECLARE @day_ahead INT = 7-(@week_day_of_reservation_date-2)
                
                IF(CAST(DATEADD(day,-@day_ahead,@target_date) AS DATE)< CAST(@reservation_date AS DATE))
                    THROW 50000, N'Jest za późno aby przyjąć zamówienie na owoce na morza na dany termin.', 1
            END
            ELSE
                THROW 50000, N'Zamówienie na owoce morze, można składać tylko w czwartek, piątek lub sobote.', 1
        END

        -- Sprawdzanie czy na ten moment jest wystarczająco składników by wykonać zamówienie
        DECLARE @available_quan INT = (SELECT available_today FROM Products WHERE product_id = @product_id)
       
        IF( @available_quan>= @quantity)
        BEGIN 
            UPDATE Products
            SET available_today = (@available_quan - @quantity)
            WHERE product_id = @product_id
        END
        ELSE
            THROW 50000, N'Na ten moment nie ma wystarczająco składników na dane zamówienie', 1
        
        -- Sprawdzenie, czy podany produkt pochodzi z menu obowiązującego na datę rezerwacji
        SET @menu_id = (SELECT menu_id FROM Menu WHERE start_date <= @target_date AND end_date >= @target_date)

        IF (@product_id NOT IN (SELECT product_id FROM Menu_Details WHERE menu_id = @menu_id  ))
            THROW 50000, N'Podany produkt nie pochodzi z menu obowiązującego na datę rezerwacji!', 1
        
        SET @price = (SELECT price FROM Menu_Details WHERE menu_id = @menu_id AND product_id = @product_id)

        INSERT INTO Reservation_Details(reservation_id,product_id,quantity,price)
        VALUES(@reservation_id,@product_id,@quantity,@price)

        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: check_company_turnover
-- Procedura podaje dochód osiągnięty przez restaurację w podanym przedziale czasowym
CREATE PROCEDURE check_company_turnover @beginning_date DATE, @end_date DATE  
AS
    SELECT SUM(price*quantity) 
    FROM Reservation_Details as RD  
    INNER JOIN Reservations as R on R.reservation_id = RD.reservation_id  
    WHERE R.reservation_date > @beginning_date AND R.reservation_date <= @end_date  
GO

-- Procedure: create_menu
-- Procedura przyjmuje datę rozpoczęcia i zakończenia obowiązywania menu wraz z listą produktów, a następnie tworzy takie menu
CREATE TYPE Products_List AS TABLE (product_id INT, price MONEY)
GO

CREATE PROCEDURE create_menu @start_date DATE, @end_date DATE, @products_list Products_List READONLY
AS
    INSERT INTO Menu(start_date, end_date)
    VALUES (@start_date, @end_date)

    DECLARE @menu_id INT = @@IDENTITY

    INSERT INTO Menu_Details(menu_id, product_id, price, is_active)
    SELECT @menu_id, product_id, price, 1 FROM @products_list
GO

-- Procedure: create_invoice
-- Procedura przyjmuje listę rezerwacji, a następnie tworzy nową fakturę i dopisuje numer tej faktury do wskazanych rezerwacji
CREATE TYPE Reservations_List AS TABLE (reservation_id INT)
GO

CREATE PROCEDURE create_invoice @reservations_list Reservations_List READONLY
AS
    BEGIN TRAN
    BEGIN TRY
        -- Sprawdzenie, czy rezerwacje pochodzą od tego samego klienta
        DECLARE @client_ids_quantity INT
        SELECT @client_ids_quantity = COUNT(*) FROM (
            SELECT DISTINCT client_id FROM Reservations AS R
            INNER JOIN @reservations_list as RL ON RL.reservation_id = R.reservation_id
        ) AS T

        IF @client_ids_quantity > 1
            THROW 50000, N'Rezerwacje nie pochodzą od tego samego klienta!', 1

        -- Sprawdzenie, czy podane rezerwacje nie posiadają jeszcze przypisanej faktury
        DECLARE @reservations_quantity INT
        SELECT @reservations_quantity = COUNT(*) FROM @reservations_list

        DECLARE @without_quantity INT
        SELECT @without_quantity = COUNT(*) FROM (
            SELECT R.invoice_id FROM Reservations AS R
            INNER JOIN @reservations_list AS RL ON RL.reservation_id = R.reservation_id
            WHERE R.invoice_id IS NULL
        ) AS T

        IF @reservations_quantity != @without_quantity
            THROW 50000, N'Co najmniej 1 z podanych rezerwacji ma już przypisaną fakturę!', 1

        -- Deklaracja i definicja zmiennej @address, @postcode oraz @city_id
        DECLARE @client_id INT
        SELECT @client_id = client_id FROM (
            SELECT TOP 1 R.client_id FROM Reservations AS R
            INNER JOIN @reservations_list AS RL ON RL.reservation_id = R.reservation_id
        ) AS T

        DECLARE @address VARCHAR(30)
        DECLARE @postcode VARCHAR(10)
        DECLARE @city_id INT

        SELECT @address = address, @postcode = postcode, @city_id = city_id FROM Clients WHERE client_id = @client_id

        -- Stworzenie unikalnego numeru faktury
        DECLARE @invoice_number varchar(30) = CONCAT(FORMAT(MONTH(GETDATE()),'0#'), '/', YEAR(GETDATE()), ' - ');

        DECLARE @invoices_quantity INT
        SELECT @invoices_quantity = COUNT(*) FROM Invoices
        WHERE invoice_number LIKE CONCAT('%', @invoice_number, '%')

        SET @invoice_number = CONCAT(@invoice_number, @invoices_quantity + 1)

        -- Deklaracja i definicja zmiennej @created oraz @is_payed
        DECLARE @created DATETIME = GETDATE()
        DECLARE @is_payed BIT = 0

        -- Zapis danych do tabeli [Invoices]
        INSERT INTO Invoices (invoice_number, created, is_payed, address, postcode, city_id)
        VALUES (@invoice_number, @created, @is_payed, @address, @postcode, @city_id)

        DECLARE @invoice_id INT = @@IDENTITY

        -- Dopisanie numeru utworzonej faktury do podanych rezerwacji w tabeli [Reservations]
        UPDATE Reservations
        SET invoice_id = @invoice_id
        WHERE reservation_id IN (SELECT * FROM @reservations_list)

        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: create_monthly_invoice
-- Procedura tworzy fakturę zawierającą wszystkie nieopłacone rezerwacje dla podanego klienta w określonym miesiącu i roku
CREATE PROCEDURE create_monthly_invoice @client_id INT, @month INT, @year INT
AS
    BEGIN TRAN
    BEGIN TRY
        -- Deklaracja i definicja zmiennej @reservations_list jako listy nieopłaconych rezerwacji klienta
        DECLARE @reservations_list Reservations_List

        INSERT INTO @reservations_list (reservation_id)
        SELECT reservation_id FROM Reservations
        WHERE client_id = @client_id AND YEAR(target_date) = @year AND MONTH(target_date) = @month AND invoice_id IS NULL

        -- Utworzenie nowej miesięcznej faktury
        EXEC create_invoice @reservations_list

        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO

-- Procedure: show_valid_menu
-- Procedura pokazuje menu, które obowiązywało na podany dzień
CREATE PROCEDURE show_valid_menu @date DATE
AS
    BEGIN TRY
        -- Deklaracja i definicja zmiennej @menu_id jako ID menu, które obowiązywało na dany dzień
        DECLARE @menu_id INT

        SELECT @menu_id = menu_id FROM Menu
        WHERE @date >= start_date AND @date <= end_date

        -- Sprawdzenie, czy istnieje takie menu
        IF @menu_id IS NULL
            THROW 50000, N'Na podany dzień nie obowiązywało żadne menu!', 1

        -- Wyświetlenie menu, które obowiązywało
        SELECT MD.menu_id, MD.product_id, P.product_name, MD.price, MD.is_active, P.is_sea_food, P.available_today, P.available_every_day FROM Menu_Details AS MD
        INNER JOIN Products AS P ON P.product_id = MD.product_id
        WHERE MD.menu_id = @menu_id
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
    END CATCH
GO

-- Procedure: create_detailed_reservation
-- Procedura dodaje rezerwację razem ze szczegółami zamówienia
CREATE TYPE Reservation_Details_List AS TABLE (product_id INT, quantity INT)
GO

CREATE PROCEDURE create_detailed_reservation @target_date DATETIME, @eating_time DATETIME, @people_count INT, @client_id INT, @last_name_list Name_List READONLY, @table_list Table_List READONLY, @reservation_details_list Reservation_Details_List READONLY 
AS
    BEGIN TRAN
    BEGIN TRY
        -- Dodanie nowej rezerwacji
        EXEC add_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list

        DECLARE @reservation_id INT = IDENT_CURRENT('Reservations')

        -- Dodanie szczegółów rezerwacji
        DECLARE @sql NVARCHAR(max) = ''

        SELECT @sql += 'EXEC add_reservation_details @reservation_id = ' + CAST(@reservation_id AS VARCHAR) + ', @product_id = ' + CAST(product_id AS VARCHAR) + ', @quantity = ' + CAST(quantity AS VARCHAR) + '; '
        FROM @reservation_details_list

        EXEC sp_executesql @sql

        COMMIT
    END TRY
    BEGIN CATCH
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
        ROLLBACK TRAN
    END CATCH
GO


-- Procedure: close_reservation
-- Procedura zamyka otwartą rezerwację
CREATE PROCEDURE close_reservation @reservation_id INT
AS
    IF (SELECT actual_eating_time FROM Reservations WHERE reservation_id = @reservation_id) is NULL
        UPDATE Reservations
        SET actual_eating_time = GETDATE()
        WHERE reservation_id = @reservation_id
    ELSE
        SELECT ERROR_PROCEDURE() AS ErrorProcedure, ERROR_LINE() AS ErrorLine, ERROR_MESSAGE() AS ErrorMessage
GO


-- Procedure: compare_menus
-- Procedura porównuje dwa menu, czy między menu1 a menu2 zmieniła się conajmniej połowa pozycji
CREATE PROCEDURE compare_menus @menu_id1 INT, @menu_id2 INT
AS

    IF((SELECT COUNT(menu_id) FROM Menu WHERE menu_id = @menu_id1) = 0)
        THROW 51000, 'menu1 does not exist.', 1;  

    IF((SELECT COUNT(menu_id) FROM Menu WHERE menu_id = @menu_id2) = 0)
        THROW 51000, 'menu2 does not exist.', 1;  


    DECLARE @menu1_size INT = (SELECT COUNT(product_id) FROM Menu_Details WHERE menu_id = @menu_id1)
    DECLARE @common_size INT = (SELECT COUNT(M1.product_id)  
    FROM
    (SELECT * FROM Menu_Details WHERE menu_id = @menu_id1) as M1
    JOIN
    (SELECT * FROM Menu_Details WHERE menu_id = @menu_id2) as M2
    ON M1.product_id = M2.product_id)

    SELECT CASE WHEN @menu1_size/2 > @common_size
                        THEN 1
                        ELSE 0
                        END
GO


Dodawanie danych

In [ ]:
-- sample data
-- Sample data: Countries  
INSERT INTO Countries (country_name)  
VALUES 
('Polska'),
('Wielka Brytania'),
('Niemcy')

-- Sample data: Cities  
INSERT INTO Cities (city_name, country_id)  
VALUES 
('Kraków', 1),
('Warszawa', 1),
('Tarnów', 1),
('Wieliczka', 1),
('Londyn', 2),
('Berlin', 3)

-- Sample data: Tables  
INSERT INTO Tables (capacity, is_active)  
VALUES  
(2, 1),  
(3, 1),  
(2, 1),
(4,5)

-- Sample data: Individual_Clients  
DECLARE @email varchar(30) = 'jkowalski@gmail.com'
DECLARE @phone varchar(20) = '123456789'
DECLARE @first_name varchar(20) = 'Jan'
DECLARE @last_name varchar(20) = 'Kowalski'
DECLARE @address varchar(30) = 'ul. Młoda 25'
DECLARE @postcode varchar(10) = '30-100'
DECLARE @city_id int = 2

EXECUTE add_individual_client @email, @phone, @first_name, @last_name, @address, @postcode, @city_id
GO

-- Sample data: Individual_Clients  
DECLARE @email varchar(30) = 'porlowski@gmail.com'
DECLARE @phone varchar(20) = '333222333'
DECLARE @first_name varchar(20) = 'Paweł'
DECLARE @last_name varchar(20) = 'Orłowski'
DECLARE @address varchar(30) = 'ul. Długa 12B'
DECLARE @postcode varchar(10) = '18-341'
DECLARE @city_id int = 1

EXECUTE add_individual_client @email, @phone, @first_name, @last_name, @address, @postcode, @city_id
GO

-- Sample data: Individual_Clients  
DECLARE @email varchar(30) = 'mnowak@gmail.com'
DECLARE @phone varchar(20) = '234345456'
DECLARE @first_name varchar(20) = 'Magdalena'
DECLARE @last_name varchar(20) = 'Nowak'
DECLARE @address varchar(30) = 'ul. Komorowskiego 123/6'
DECLARE @postcode varchar(10) = '32-384'
DECLARE @city_id int = 1

EXECUTE add_individual_client @email, @phone, @first_name, @last_name, @address, @postcode, @city_id
GO

-- Sample data: Company Clients
DECLARE @email varchar(30) = 'GymManiacs@gmail.com'
DECLARE @phone varchar(20) = '234084768'
DECLARE @name varchar(20) = 'Gym Maniacs'
DECLARE @NIP varchar(15) = '8756701430'
DECLARE @address varchar(30) = 'The Ridgeway 160'
DECLARE @postcode varchar(10) = '10160'
DECLARE @city_id int = 5

EXEC add_company_client @email, @phone, @name, @NIP, @address, @postcode, @city_id
GO

-- Sample data: Company Clients
DECLARE @email varchar(30) = 'bestdonuts@gmail.com'
DECLARE @phone varchar(20) ='464575686'
DECLARE @name varchar(20) = 'Donuts'
DECLARE @NIP varchar(15) = '1325476980'
DECLARE @address varchar(30) = 'Kantstrabe 51'
DECLARE @postcode varchar(10) = '10625'
DECLARE @city_id int = 6

EXEC add_company_client @email, @phone, @name, @NIP, @address, @postcode, @city_id
GO

-- Sample data: Company Clients
DECLARE @email varchar(30) = 'fastcars@gmail.com'
DECLARE @phone varchar(20) ='233345091'
DECLARE @name varchar(20) = 'Fast Cars'
DECLARE @NIP varchar(15) = '1324234678'
DECLARE @address varchar(30) = 'Armii Krajowej 20'
DECLARE @postcode varchar(10) = '30-150'
DECLARE @city_id int = 1

EXEC add_company_client @email, @phone, @name, @NIP, @address, @postcode,  @city_id
GO 

-- Sample data: Discounts
INSERT INTO Discounts (client_id, discount, start_date, end_date)
VALUES (4, 0.25, '20220110', '20220210')

-- Sample data: Products 
INSERT INTO Products (product_name, is_sea_food, available_every_day, available_today) 
VALUES 
('Kotlet mielony', 0, 30, 30), 
('Zupa pomidorowa', 0, 50, 50), 
('Coca-Cola', 0, 100, 80), 
('Tiramisu', 0, 10, 10), 
('Pizza hawajska', 0, 35, 35), 
('Woda gazowana', 0, 100, 70),
('Woda niegazowana', 0, 120, 120)

-- Sample data: Menu and Menu_Details
DECLARE @products_list Products_List
INSERT @products_list VALUES (2, 12), (3, 5), (1, 18), (4, 10), (5, 20), (6, 3)

DECLARE @start_date date = '20220105'
DECLARE @end_date date = '20220115'

EXEC create_menu @start_date, @end_date, @products_list
GO

DECLARE @products_list Products_List
INSERT @products_list VALUES (2, 12), (3, 5), (1, 18), (4, 10), (5, 22), (7, 2)

DECLARE @start_date date = '20220116'
DECLARE @end_date date = '20220126'

EXEC create_menu @start_date, @end_date, @products_list
GO

-- Sample data: Reservations
DECLARE @target_date datetime = '20220107 01:00 PM'
DECLARE @eating_time datetime = '20220107 03:00 PM'
DECLARE @people_count int = 4
DECLARE @client_id int = 3
DECLARE @last_name_list [dbo].[Name_List]
DECLARE @table_list [dbo].[Table_List]
INSERT INTO @last_name_list VALUES ('Nowak'),('Kowalsky')
INSERT INTO @table_list VALUES (1),(3)

EXEC add_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list
GO

-- Sample data: Reservations
DECLARE @target_date datetime = '20220106 3:40 PM'
DECLARE @eating_time datetime = '20220106 5:40 PM'
DECLARE @people_count int = 2
DECLARE @client_id int = 3
DECLARE @last_name_list [dbo].[Name_List]
DECLARE @table_list [dbo].[Table_List]
INSERT INTO @last_name_list VALUES ('Poręba')
INSERT INTO @table_list VALUES (1)

EXEC add_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list
GO

-- Sample data: Reservations
DECLARE @target_date datetime = '20220108 2:20 PM'
DECLARE @eating_time datetime = '20220108 4:20 PM'
DECLARE @people_count int = 3
DECLARE @client_id int = 1
DECLARE @last_name_list [dbo].[Name_List]
DECLARE @table_list [dbo].[Table_List]
INSERT INTO @table_list VALUES (2)

EXEC add_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list
GO

-- Sample data: Reservations
DECLARE @target_date datetime = '20220108 1:00 PM'
DECLARE @eating_time datetime = '20220108 3:00 PM'
DECLARE @people_count int = 5
DECLARE @client_id int = 2
DECLARE @last_name_list [dbo].[Name_List]
DECLARE @table_list [dbo].[Table_List]
INSERT INTO @table_list VALUES (3), (4)

EXEC add_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list
GO

-- Sample data: Reservation_Details
DECLARE @reservation_id INT = 1
DECLARE @product_id INT = 2
DECLARE @quantity INT = 2

EXEC add_reservation_details @reservation_id, @product_id, @quantity  
GO

-- Sample data: Reservation_Details
DECLARE @reservation_id INT = 1
DECLARE @product_id INT = 3
DECLARE @quantity INT = 2

EXEC add_reservation_details @reservation_id, @product_id, @quantity  
GO

-- Sample data: Reservation_Details
DECLARE @reservation_id INT = 2
DECLARE @product_id INT = 1
DECLARE @quantity INT = 1

EXEC add_reservation_details @reservation_id, @product_id, @quantity  
GO

-- Sample data: Reservation_Details
DECLARE @reservation_id INT = 2
DECLARE @product_id INT = 3
DECLARE @quantity INT = 1

EXEC add_reservation_details @reservation_id, @product_id, @quantity  
GO

-- Sample data: Reservation_Details
DECLARE @reservation_id INT = 3
DECLARE @product_id INT = 4
DECLARE @quantity INT = 5

EXEC add_reservation_details @reservation_id, @product_id, @quantity  
GO

-- Sample data: Invoices
DECLARE @client_id INT = 1
DECLARE @month INT = 1
DECLARE @year INT = 2022

EXEC create_monthly_invoice @client_id, @month, @year
GO

-- Sample data: Reservations and Reservation_Details
DECLARE @target_date datetime = '20220110 3:00 PM'
DECLARE @eating_time datetime = '20220110 6:00 PM'
DECLARE @people_count int = 1
DECLARE @client_id int = 5
DECLARE @last_name_list Name_List
DECLARE @table_list Table_List
DECLARE @reservation_details_list Reservation_Details_List
INSERT INTO @table_list VALUES (4)
INSERT INTO @reservation_details_list VALUES (1, 3), (2, 1), (3, 2)

EXEC create_detailed_reservation @target_date, @eating_time, @people_count, @client_id, @last_name_list, @table_list, @reservation_details_list 
GO

Dodawanie constraint'ów do tabel na sprawdzenie dat

In [ ]:
-- date constraints
-- Date constraint: Discounts
ALTER TABLE Discounts WITH NOCHECK ADD CONSTRAINT Discounts_ck_start_date
    CHECK (start_date >= GETDATE());

-- Date constraint: Menu
ALTER TABLE Menu WITH NOCHECK ADD CONSTRAINT Menu_ck_start_date 
    CHECK (start_date >= GETDATE());

-- Date constraint: Reservations
ALTER TABLE Reservations WITH NOCHECK ADD CONSTRAINT Reservations_ck_reservation_date
    CHECK (reservation_date >= GETDATE());

-- Date constraint: Reservations
ALTER TABLE Reservations WITH NOCHECK ADD CONSTRAINT Reservations_ck_target_date 
    CHECK (target_date >= GETDATE());

Tworzenie widoków

In [ ]:
-- views
-- View: individual_clients_report
-- Widok pokazuje sumaryczną kwotę zamówień od każdego klienta indywidualnego, który zamawiał w restauracji posiłek
CREATE VIEW individual_clients_report 
AS
    SELECT IC.client_id, IC.first_name, IC.last_name, ISNULL(sum(RD.price * RD.quantity), 0) as order_sum  
    FROM Individual_Clients AS IC  
    JOIN Clients as C ON IC.client_id = C.client_id  
    LEFT OUTER JOIN Reservations AS R on R.client_id = C.client_id  
    LEFT OUTER JOIN Reservation_Details AS RD on R.reservation_id = RD.reservation_id  
    GROUP BY Ic.client_id, IC.last_name, IC.first_name
GO

-- View: company_clients_report
-- Widok pokazuje sumaryczną kwotę zamówień od każdego klienta biznesowego, który zamawiał w restauracji posiłek
CREATE VIEW company_clients_report 
AS  
    SELECT CC.client_id, CC.name, ISNULL(sum(RD.price * RD.quantity), 0) as order_sum  
    FROM Company_Clients AS CC  
    LEFT OUTER JOIN Reservations AS R on R.client_id = CC.client_id  
    LEFT OUTER JOIN Reservation_Details as RD on R.reservation_id = RD.reservation_id  
    GROUP BY CC.client_id, CC.name
GO

-- View: product_stats
-- Widok pokazuje sumaryczną liczbę zamówień każdego z produktów, które kiedykolwiek były sprzedawane w restauracji
CREATE VIEW product_stats 
AS 
    SELECT P.product_id, P.product_name, ISNULL(sum(RD.quantity), 0) as ordered_count, ISNULL(sum(RD.quantity*RD.price), 0) as income 
    FROM Products as P 
    LEFT OUTER JOIN Reservation_Details as RD on P.product_id = RD.product_id 
    GROUP BY P.product_id, P.product_name
GO

-- View: current_menu
-- Widok pokazuje obecnie obowiązujące menu
CREATE VIEW current_menu 
AS  
    SELECT product_name, md.price FROM Products as p  
    INNER JOIN Menu_Details as md on md.product_id = p.product_id  
    INNER JOIN Menu as m on m.menu_id = md.menu_id  
    WHERE start_date >= GETDATE() AND end_date <= GETDATE()
GO

-- View: discounts_individual_clients
-- Widok wyświetla klientów indywidualnych, którzy w tym momencie posiadają jakąś zniżkę
CREATE VIEW discounts_individual_clients
AS
    SELECT ic.first_name, ic.last_name, c.email, c.phone, isnull(discount,0) as discount FROM Discounts as d
    right outer join Clients  as c on c.client_id = d.client_id
    inner join Individual_clients as ic on ic.client_id = c.client_id
    where d.start_date <= cast(GETDATE() as DATE) AND d.end_date >= cast(GETDATE() as DATE)
GO

-- View: discounts_company_clients
-- Widok wyświetla klientów biznesowych, którzy w tym momencie posiadają jakąś zniżkę
CREATE VIEW discounts_company_clients
AS 
    SELECT cc.name, c.email, c.phone, isnull(discount,0) as discount FROM Discounts as d
    right outer join Clients  as c on c.client_id = d.client_id
    inner join Company_clients as cc on cc.client_id = c.client_id
    where d.start_date <= cast(GETDATE() as DATE) AND d.end_date >= cast(GETDATE() as DATE)
GO

-- View: table_weekly_raport
-- Widok prezentuje ilość rezerwacji dla każdego ze stolików w restauracji
CREATE VIEW table_weekly_raport
AS
    select table_id, count(*) as number from Reservation_Tables as rt
    inner join Reservations as r on r.reservation_id = rt.reservation_id
    where target_date >= GETDATE()-7 and target_date <= GETDATE()
    group by table_id
GO

Tworzenie ról i uprawnień

In [ ]:
-- roles
CREATE ROLE RESTAURANT_OWNER;
CREATE ROLE MANAGER;
CREATE ROLE CHEF;
CREATE ROLE WAITER;
CREATE ROLE CLIENT;

-- Table: Clients
GRANT SELECT, INSERT, UPDATE ON Clients TO WAITER;

-- Table: Individual_Clients
GRANT SELECT, INSERT, UPDATE ON Individual_Clients TO WAITER;

-- Table: Company_Clients
GRANT SELECT, INSERT, UPDATE ON Company_Clients TO WAITER;

-- Table: Countries
GRANT SELECT, INSERT, UPDATE ON Countries TO WAITER;

-- Table: Cities
GRANT SELECT, INSERT, UPDATE ON Cities TO WAITER;

-- Table: Discounts
GRANT SELECT, INSERT, UPDATE ON Discounts TO WAITER;

-- Table: Products
GRANT SELECT, INSERT, UPDATE ON Products TO MANAGER;
GRANT SELECT, UPDATE ON Products TO WAITER;
GRANT SELECT ON Products TO CLIENT;
GRANT SELECT ON Products TO CHEF;

-- Table: Menu
GRANT SELECT, INSERT, UPDATE ON Menu TO MANAGER;
GRANT SELECT ON Menu TO WAITER;
GRANT SELECT ON Menu TO CLIENT;
GRANT SELECT ON Menu TO CHEF;

-- Table: Menu_Details
GRANT SELECT, INSERT, UPDATE ON Menu_Details TO MANAGER;
GRANT SELECT ON Menu_Details TO WAITER;
GRANT SELECT ON Menu_Details TO CLIENT;
GRANT SELECT ON Menu_Details TO CHEF;

-- Table: Reservation_Details
GRANT SELECT, INSERT, UPDATE ON Reservation_Details TO WAITER;
GRANT SELECT ON Reservation_Details TO CHEF;

-- Table: Reservation_People
GRANT SELECT, INSERT, UPDATE ON Reservation_People TO WAITER;

-- Table: Reservation_Tables
GRANT SELECT, INSERT, UPDATE ON Reservation_Tables TO WAITER;

-- Table: Reservations
GRANT SELECT, INSERT, UPDATE ON Reservations TO WAITER;

-- Table: Tables
GRANT INSERT, UPDATE ON Tables TO MANAGER;
GRANT SELECT ON Tables TO WAITER;

-- Table: Invoices
GRANT UPDATE ON Invoices TO MANAGER;
GRANT SELECT, INSERT ON Invoices TO WAITER;

-- Procedure: add_individual_client
GRANT EXECUTE ON add_individual_client TO WAITER;

-- Procedure: add_company_client
GRANT EXECUTE ON add_company_client TO WAITER;

-- Procedure: tables_available
GRANT EXECUTE ON tables_available TO WAITER

-- Procedure: add_reservation
GRANT EXECUTE ON add_reservation TO WAITER

-- Procedure: add_reservation_details
GRANT EXECUTE ON add_reservation_details TO WAITER

-- Procedure: check_company_turnover
GRANT EXECUTE ON check_company_turnover TO MANAGER

-- Procedure: create_menu
GRANT EXECUTE ON create_menu TO MANAGER

-- Procedure: create_invoice
GRANT EXECUTE ON create_invoice TO WAITER

-- Procedure: create_monthly_invoice
GRANT EXECUTE ON create_monthly_invoice TO WAITER

-- Procedure: show_valid_menu
GRANT EXECUTE ON show_valid_menu TO WAITER

-- Procedure: create_detailed_reservation
GRANT EXECUTE ON create_detailed_reservation TO WAITER

-- Procedure: close_reservation
GRANT EXECUTE ON close_reservation TO WAITER

-- Procedure: compare_menus
GRANT EXECUTE ON compare_menus TO MANAGER

-- View: individual_clients_report
GRANT SELECT ON individual_clients_report TO MANAGER

-- View: company_clients_report
GRANT SELECT ON company_clients_report TO MANAGER

-- View: product_stats
GRANT SELECT ON product_stats TO MANAGER

-- View: current_menu
GRANT SELECT ON current_menu TO WAITER

-- View: discounts_individual_clients
GRANT SELECT ON discounts_individual_clients TO MANAGER

-- View: discounts_company_clients
GRANT SELECT ON discounts_company_clients TO MANAGER

-- View: table_weekly_raport
GRANT SELECT ON table_weekly_raport TO MANAGER

Tworzenie indeksów

In [ ]:
-- indexes
-- Index: MenuIndex
CREATE INDEX MenuIndex
ON Menu (start_date,end_date)

-- Index: CityIndex
CREATE INDEX CityIndex
ON Cities (city_name)

-- Index: ProductsIndex
CREATE INDEX ProductsIndex
ON Products (product_name)